In [ ]:
## For debbugging
%load_ext autoreload

In [13]:
# import os
import sys
from pathlib import Path
module_path = Path('.').resolve().parents[0]
if module_path not in sys.path:
    sys.path.append(str(module_path))


In [11]:
## for debugging:
%aimport mlp.keyword
%aimport mlp.tools
%aimport mlp.tree
%autoreload 1
from mlp.keyword import *

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tqdm
%matplotlib inline

import spacy, textacy, platform
print(
    f'Textacy: \t{textacy.__version__:}'+\
    f'\nspaCy: \t{spacy.__version__:>13}'+\
    f'\nPython: \t{platform.python_version()}'
    )

Textacy: 	0.5.0
spaCy: 	        2.0.4
Python: 	3.6.3


In [148]:
import dask.dataframe as dd
import dask
import string

nlp = spacy.load('en')
# data_dir = module_path/'data'/'mine_data'
# df = dd.read_csv(data_dir/'loader_work_orders_sanitised.csv')
data_dir = module_path/'data'/'gs_data'
df = pd.read_csv(data_dir/'Maintenance_All_Clean.csv')

In [153]:
raw_text = df.Description.astype(str) + '. ' + df.Resolution.astype(str)
# raw_text = df.ShortText.astype(str)
raw_text = raw_text.fillna('')
raw_text = raw_text.str.lower()
# raw_text.str.replace('\n', ' ')
# raw_text = raw_text.str.replace('[{}]'.format(string.punctuation), '')

def text_preproc(doc, **preproc_kws):
#     x = textacy.preprocess.normalize_whitespace(x)
#     x = textacy.preprocess.preprocess_text(x, **preproc_kws)
#     doc = nlp(x)
    proc_list = [token.lemma_ for token in doc
                   if not (punct_space(token) or token.is_stop)]
    proc_doc = ' '.join(proc_list)
    if len(proc_doc) <= 1:
        proc_doc = 'NaN'
    return proc_doc

def series_itervals(s):
    for n, val in s.iteritems():
        yield val
for doc in tqdm(nlp.pipe(series_itervals(raw_text))):
    new_doc = text_preproc(doc)
# raw_text = raw_text.apply(textacy.preprocess.normalize_whitespace, meta=('RawText', str))
# preproc_kws = {
#     'fix_unicode': True, 
#     'lowercase': True,
#     'no_urls': True,
#     'no_emails': True,
#     'no_phone_numbers': True,
#     'no_punct': True,
#     'no_contractions': True,
#     'no_accents': True
# }
# raw_text = raw_text.apply(textacy.preprocess.preprocess_text, meta=raw_text, **preproc_kws)
# raw_text = raw_text.apply(text_preproc, meta=raw_text, **preproc_kws)
# tqdm.pandas(desc="KEx")
# raw_text = raw_text.progress_apply(text_preproc)
# df['RawText'] = raw_text

# with dask.set_options(get=dask.multiprocessing.get):
# raw_text.to_csv(data_dir/'TEMP_init-*.txt', index=False)


# with dask.set_options(get=dask.multiprocessing.get):
#     rt = raw_text.compute()
# rt

3435it [00:10, 335.10it/s]


In [113]:
for fname in data_dir.glob('TEMP_init-[!0]*.txt'):
    print(fname)

In [129]:
# docs = textacy.fileio.read.read_csv(str(data_dir/'TEMP_init-0.txt'))
# Descriptions in RawText col
# content_stream, metadata_stream = textacy.fileio.split_record_fields(docs, 1)  

corpus = textacy.Corpus(u'en')

for fname in data_dir.glob('TEMP_init-*.txt'):
    print(fname)
    docs = textacy.fileio.read.read_csv(str(fname), delimiter='\t')
    content_stream, _ = textacy.fileio.split_record_fields(docs, 0)  
    corpus.add_texts(content_stream)

/home/tbsexton/Documents/Projects/DiagnosticKB/MaintenanceLanguageProcessing/data/gs_data/TEMP_init-0.txt


In [144]:
vsm = textacy.vsm.Vectorizer(weighting='tfidf',
                             normalize=False,
                             smooth_idf=False,
                             sublinear_tf=True,
                             min_df=2, max_df=0.95)  # each token in >2 docs, <95% of docs

terms_list = (doc.to_terms_list(ngrams=(1, 2, 3),
                                normalize=u'lemma',
                                named_entities=False,
                                filter_stops=True,  # Nope! Not needed :)
                                filter_punct=True,
                                as_strings=True) for doc in corpus)
doc_term_mat = vsm.fit_transform(terms_list)
doc_term_mat

def get_norm_tokens(doc_n, doc_term_mat, id_to_term):
    doc = doc_term_mat[doc_n].toarray()
    # we only care about unique tags, right?
    return set([id_to_term[i] for i in doc.nonzero()[1]])

# list(list(terms_list)[1])

In [145]:
get_norm_tokens(0, doc_term_mat, vsm.id_to_term)

{'bar',
 'bar feeder',
 'bar feeder chain',
 'break',
 'break bar',
 'break bar feeder',
 'chain',
 'feeder',
 'feeder chain',
 'repair'}

In [ ]:
vsm.sublinear_tf()